# सेमांटिक कर्नेल

इस कोड उदाहरण में, आप [सेमांटिक कर्नेल](https://aka.ms/ai-agents-beginners/semantic-kernel) एआई फ्रेमवर्क का उपयोग करके एक बुनियादी एजेंट बनाएंगे।

इस उदाहरण का उद्देश्य आपको उन चरणों को दिखाना है, जिन्हें हम बाद के कोड उदाहरणों में विभिन्न एजेंटिक पैटर्न को लागू करते समय उपयोग करेंगे।


## आवश्यक पायथन पैकेज आयात करें


In [ ]:
import json
import os 

from typing import Annotated

from dotenv import load_dotenv

from IPython.display import display, HTML

from openai import AsyncOpenAI

from semantic_kernel.agents import ChatCompletionAgent, ChatHistoryAgentThread
from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion
from semantic_kernel.contents import FunctionCallContent, FunctionResultContent, StreamingTextContent
from semantic_kernel.functions import kernel_function

## क्लाइंट बनाना

इस उदाहरण में, हम [GitHub Models](https://aka.ms/ai-agents-beginners/github-models) का उपयोग LLM तक पहुंचने के लिए करेंगे।

`ai_model_id` को `gpt-4o-mini` के रूप में परिभाषित किया गया है। GitHub Models मार्केटप्लेस में उपलब्ध किसी अन्य मॉडल को चुनकर परिणामों में अंतर देखने की कोशिश करें।

`Azure Inference SDK` का उपयोग करने के लिए, जो GitHub Models के `base_url` के लिए उपयोग किया जाता है, हम Semantic Kernel के भीतर `OpenAIChatCompletion` कनेक्टर का उपयोग करेंगे। Semantic Kernel को अन्य मॉडल प्रदाताओं के लिए उपयोग करने के लिए अन्य [उपलब्ध कनेक्टर्स](https://learn.microsoft.com/semantic-kernel/concepts/ai-services/chat-completion) भी हैं।


In [ ]:
import random   

# Define a sample plugin for the sample

class DestinationsPlugin:
    """A List of Random Destinations for a vacation."""

    def __init__(self):
        # List of vacation destinations
        self.destinations = [
            "Barcelona, Spain",
            "Paris, France",
            "Berlin, Germany",
            "Tokyo, Japan",
            "Sydney, Australia",
            "New York, USA",
            "Cairo, Egypt",
            "Cape Town, South Africa",
            "Rio de Janeiro, Brazil",
            "Bali, Indonesia"
        ]
        # Track last destination to avoid repeats
        self.last_destination = None

    @kernel_function(description="Provides a random vacation destination.")
    def get_random_destination(self) -> Annotated[str, "Returns a random vacation destination."]:
        # Get available destinations (excluding last one if possible)
        available_destinations = self.destinations.copy()
        if self.last_destination and len(available_destinations) > 1:
            available_destinations.remove(self.last_destination)

        # Select a random destination
        destination = random.choice(available_destinations)

        # Update the last destination
        self.last_destination = destination

        return destination

In [ ]:
load_dotenv()
client = AsyncOpenAI(
    api_key=os.environ.get("GITHUB_TOKEN"), 
    base_url="https://models.inference.ai.azure.com/",
)

# Create an AI Service that will be used by the `ChatCompletionAgent`
chat_completion_service = OpenAIChatCompletion(
    ai_model_id="gpt-4o-mini",
    async_client=client,
)

## एजेंट बनाना

नीचे हम `TravelAgent` नामक एजेंट बनाते हैं।

इस उदाहरण में, हम बहुत ही सरल निर्देशों का उपयोग कर रहे हैं। आप इन निर्देशों को बदल सकते हैं यह देखने के लिए कि एजेंट अलग-अलग तरीके से कैसे प्रतिक्रिया करता है।


In [ ]:
agent = ChatCompletionAgent(
    service=chat_completion_service, 
    plugins=[DestinationsPlugin()],
    name="TravelAgent",
    instructions="You are a helpful AI Agent that can help plan vacations for customers at random destinations",
)

## एजेंट्स को चलाना

अब हम `ChatHistory` को परिभाषित करके और उसमें `system_message` जोड़कर एजेंट को चला सकते हैं। हम पहले से परिभाषित `AGENT_INSTRUCTIONS` का उपयोग करेंगे। 

इनके परिभाषित होने के बाद, हम एक `user_inputs` बनाएंगे, जो उपयोगकर्ता द्वारा एजेंट को भेजा जाने वाला संदेश होगा। इस मामले में, हमने इस संदेश को `Plan me a sunny vacation` पर सेट किया है। 

इस संदेश को बदलने के लिए स्वतंत्र महसूस करें और देखें कि एजेंट अलग-अलग तरीके से कैसे प्रतिक्रिया देता है।


In [ ]:
user_inputs = [
    "Plan me a day trip.",
    "I don't like that destination. Plan me another vacation.",
]

async def main():
    thread: ChatHistoryAgentThread | None = None

    for user_input in user_inputs:
        html_output = (
            f"<div style='margin-bottom:10px'>"
            f"<div style='font-weight:bold'>User:</div>"
            f"<div style='margin-left:20px'>{user_input}</div></div>"
        )

        agent_name = None
        full_response: list[str] = []
        function_calls: list[str] = []

        # Buffer to reconstruct streaming function call
        current_function_name = None
        argument_buffer = ""

        async for response in agent.invoke_stream(
            messages=user_input,
            thread=thread,
        ):
            thread = response.thread
            agent_name = response.name
            content_items = list(response.items)

            for item in content_items:
                if isinstance(item, FunctionCallContent):
                    if item.function_name:
                        current_function_name = item.function_name

                    # Accumulate arguments (streamed in chunks)
                    if isinstance(item.arguments, str):
                        argument_buffer += item.arguments
                elif isinstance(item, FunctionResultContent):
                    # Finalize any pending function call before showing result
                    if current_function_name:
                        formatted_args = argument_buffer.strip()
                        try:
                            parsed_args = json.loads(formatted_args)
                            formatted_args = json.dumps(parsed_args)
                        except Exception:
                            pass  # leave as raw string

                        function_calls.append(f"Calling function: {current_function_name}({formatted_args})")
                        current_function_name = None
                        argument_buffer = ""

                    function_calls.append(f"\nFunction Result:\n\n{item.result}")
                elif isinstance(item, StreamingTextContent) and item.text:
                    full_response.append(item.text)

        if function_calls:
            html_output += (
                "<div style='margin-bottom:10px'>"
                "<details>"
                "<summary style='cursor:pointer; font-weight:bold; color:#0066cc;'>Function Calls (click to expand)</summary>"
                "<div style='margin:10px; padding:10px; background-color:#f8f8f8; "
                "border:1px solid #ddd; border-radius:4px; white-space:pre-wrap; font-size:14px; color:#333;'>"
                f"{chr(10).join(function_calls)}"
                "</div></details></div>"
            )

        html_output += (
            "<div style='margin-bottom:20px'>"
            f"<div style='font-weight:bold'>{agent_name or 'Assistant'}:</div>"
            f"<div style='margin-left:20px; white-space:pre-wrap'>{''.join(full_response)}</div></div><hr>"
        )

        display(HTML(html_output))

await main()


---

**अस्वीकरण**:  
यह दस्तावेज़ AI अनुवाद सेवा [Co-op Translator](https://github.com/Azure/co-op-translator) का उपयोग करके अनुवादित किया गया है। जबकि हम सटीकता सुनिश्चित करने का प्रयास करते हैं, कृपया ध्यान दें कि स्वचालित अनुवाद में त्रुटियां या अशुद्धियां हो सकती हैं। मूल भाषा में उपलब्ध मूल दस्तावेज़ को प्रामाणिक स्रोत माना जाना चाहिए। महत्वपूर्ण जानकारी के लिए, पेशेवर मानव अनुवाद की सिफारिश की जाती है। इस अनुवाद के उपयोग से उत्पन्न किसी भी गलतफहमी या गलत व्याख्या के लिए हम जिम्मेदार नहीं हैं।
